# 

# Optimizing an Machine Learning Pipeline in Azure
## Using Hyperdrive and AutoML

### Preparing the experiment

Let's start by creating a new experiment in order to find out which of the two methods can find the best model.

In [1]:
from azureml.core import Workspace, Experiment
import os

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="bankdata-hyperdrive")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity
Azure region: eastus2
Subscription id: 79facb57-c974-4c43-935b-f645fdca73d9
Resource group: standard


We also need a compute cluster to run the training jobs on.

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException


cluster_name = "compute-cluster"

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)


Creating new cpu-cluster
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive

We use Hyperdrive to find the best parameters for our LogisticRegression model. Hyperdrive will run the **train.py** script with different hyperparameters in order to find the best model.

In [3]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive import BanditPolicy, PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        
        '--C': choice(10, 1, .1, .01),
        '--max_iter': choice(100, 200, 500, 1000)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
script_folder = "."
src = ScriptRunConfig(source_directory=script_folder,
                      script='train.py',
                      environment=sklearn_env,
                      compute_target=compute_target)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, 
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name='Accuracy', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=50,
                             max_duration_minutes=60,
                             max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Finally we get the hyperdrive run with the best metric score and save the corresponding model.

In [5]:
hyperdrive_run.wait_for_completion()


best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

for file in best_run.get_file_names():
    if file.startswith("model"):
        best_run.download_file(file, output_file_path=f"./outputs/best_hyperdrive_model.pkl")

## 2 AutoML

Let's now train a model for the same dataset using AutoML and see what AutoML can come up with.

In [6]:
exp = Experiment(workspace=ws, name="bankdata-automl")

exp.start_logging()

Experiment,Id,Type,Status,Details Page,Docs Page
bankdata-automl,5c7e05d0-3065-4bda-8102-b6bbbe0d9586,,Running,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

We reuse the clean_data function from our train.py script above.

In [8]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x["y"] = y

x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


Now we setup an automl job with cross validation and a holdout test data set.

In [9]:
from azureml.train.automl import AutoMLConfig
from azureml.core import Dataset

datastore = ws.get_default_datastore()
train_data = Dataset.Tabular.register_pandas_dataframe(
        dataframe=x, 
        name='bank-data', 
        target=datastore
    )

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="y",
    debug_log="automl_errors.log",
    compute_target=compute_target,
    n_cross_validations=5,
    test_size=.2
)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/3ce1532b-cad4-478f-9309-778258cd2425/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'education_basic.4y'

In [10]:
from azureml.widgets import RunDetails

automl_run = exp.submit(automl_config, show_output=True)

RunDetails(automl_run).show()

automl_run.wait_for_completion()

Submitting remote run.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
bankdata-automl,AutoML_25d6258e-45a2-49a0-a217-a9274d5fc222,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_25d6258e-45a2-49a0-a217-a9274d5fc222',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-02-16T13:01:47.213494Z',
 'endTimeUtc': '2023-02-16T13:37:19.588443Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"bankdata-automl","subscription_id":"79facb57-c974-4c43-935b-f645fdca73d9","resource_group":"standard","workspace_name":"udacity","region":"eastus2","compute_target":"compute-cluster","spark_service":null,"azu

Finally we find the best run and save the corresponding model.

In [11]:
best_run = automl_run.get_best_child()
best_run.download_file("outputs/model.pkl", "./outputs/best_automl_model.pkl")

## Delete the compute cluster

In [14]:
instance = ComputeTarget(workspace=ws, name=cluster_name)

try:
  instance.delete()
  instance.wait_for_completion(show_output=True)
  print('Deleted compute resource')
except ComputeTargetException:
  print('Already deleted!')

ComputeTargetException: ComputeTargetException:
	Message: ComputeTargetNotFound: Compute Target with name compute-cluster not found in provided workspace
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "ComputeTargetNotFound: Compute Target with name compute-cluster not found in provided workspace"
    }
}